In [1]:
import pandas as pd

from bodyport.load import DataWarehouseManager
from bodyport.config import EXAMPLE_ECG_DIR_LATEST
from bodyport.config import EXAMPLE_ECG_DIR_NEW
from bodyport.orm import Run, Subject

In [2]:
data_warehouse = DataWarehouseManager()

In [3]:
data_warehouse.status

'up'

In [4]:
data_warehouse.down()

data_warehouse.up()

In [6]:
# load the new data_dir into the warehouse
data_warehouse.update(data_dir=EXAMPLE_ECG_DIR_LATEST)

In [7]:
runs = data_warehouse.pandas_query('select * from run;')

runs

,id,subject_id,number,clinic_id,measurement,date,units,fs,raw_path,meta_path,processed_path,processed_at,age_at_run,sex,run_hash
0,1,12,1,sf_state,cg,2005-04-26,mV,500,/Users/sep/workspace/takehomes/bodyport/data/i...,/Users/sep/workspace/takehomes/bodyport/data/i...,None,2020-03-31 00:57:36.728195,22,female,None
1,2,12,2,sf_state,cg,2005-04-26,mV,500,/Users/sep/workspace/takehomes/bodyport/data/i...,/Users/sep/workspace/takehomes/bodyport/data/i...,None,2020-03-31 00:57:36.728195,22,female,None
2,3,15,1,sf_state,cg,2005-04-26,mV,500,/Users/sep/workspace/takehomes/bodyport/data/i...,/Users/sep/workspace/takehomes/bodyport/data/i...,None,2020-03-31 00:57:36.728195,21,female,None
3,4,15,2,sf_state,cg,2005-04-26,mV,500,/Users/sep/workspace/takehomes/bodyport/data/i...,/Users/sep/workspace/takehomes/bodyport/data/i...,None,2020-03-31 00:57:36.728195,21,female,None
4,5,23,1,sf_state,cg,2005-04-26,mV,500,/Users/sep/workspace/takehomes/bodyport/data/i...,/Users/sep/workspace/takehomes/bodyport/data/i...,None,2020-03-31 00:57:36.728195,25,male,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,284,51,4,sf_state,cg,2005-05-20,mV,500,/Users/sep/workspace/takehomes/bodyport/data/i...,/Users/sep/workspace/takehomes/bodyport/data/i...,None,2020-03-31 00:57:36.728195,31,male,None
284,285,69,1,sf_state,cg,2005-05-12,mV,500,/Users/sep/workspace/takehomes/bodyport/data/i...,/Users/sep/workspace/takehomes/bodyport/data/i...,None,2020-03-31 00:57:36.728195,16,male,None
285,286,69,2,sf_state,cg,2005-05-12,mV,500,/Users/sep/workspace/takehomes/bodyport/data/i...,/Users/sep/workspace/takehomes/bodyport/data/i...,None,2020-03-31 00:57:36.728195,16,male,None
286,287,56,1,sf_state,cg,2005-05-12,mV,500,/Users/sep/workspace/takehomes/bodyport/data/i...,/Users/sep/workspace/takehomes/bodyport/data/i...,None,2020-03-31 00:57:36.728195,21,female,None


In [8]:
runs['measurement']

0      cg
1      cg
2      cg
3      cg
4      cg
       ..
283    cg
284    cg
285    cg
286    cg
287    cg
Name: measurement, Length: 288, dtype: object

In [9]:
subjects = data_warehouse.pandas_query('select * from subject;')

subjects

,id,sex,birth_year
0,1,male,1979
1,2,female,1981
2,3,female,1982
3,4,female,1984
4,5,male,1984
...,...,...,...
75,76,female,1985
76,77,male,1986
77,78,male,1971
78,79,female,1980


In [10]:
# ensure all subjects have the same age. 
# this is as i expected for this toy example, but 
# in reality I would assume the age is calculated at `date`
# on the run, and we would want to back out approximate birth year from it
# to put into the subject table
assert runs.groupby('subject_id')['age_at_run'].nunique().nunique() == 1

In [11]:
# ensure all subjects have same sex in each run
assert runs.groupby('subject_id')['sex'].nunique().nunique() == 1

In [15]:
# now assume there's new incoming data:
data_warehouse.update(data_dir=EXAMPLE_ECG_DIR_NEW)

In [16]:
# we see the new subject (81) is in the subject table
data_warehouse.pandas_query('select * from subject;')

,id,sex,birth_year
0,1,male,1979
1,2,female,1981
2,3,female,1982
3,4,female,1984
4,5,male,1984
...,...,...,...
76,77,male,1986
77,78,male,1971
78,79,female,1980
79,80,female,1985


In [ ]:
data_warehouse.down()